In [1]:
import pandas as pd
import numpy as np
import itertools
from xml.etree.ElementTree import Element, SubElement, tostring
import datetime

In [2]:
%matplotlib inline
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
def datetime_to_secs(time):  
    if type(time) is datetime.time:
        secs = time.hour * 3600 + time.minute * 60 + time.second 
        return secs
    else:
        print('Provide a datetime.time var')

In [4]:
def randNormal_time(time, dev):
    mean = datetime_to_secs(datetime.datetime.strptime(time, '%H:%M').time())
    std = datetime_to_secs(datetime.datetime.strptime(dev, '%H:%M').time())
    # Draw from a normal distr
    time_secs = np.random.normal(loc=mean, scale=std)
    rand_time = str(datetime.timedelta(seconds=time_secs)).split('.')[0]
    return rand_time

In [5]:
class Agent(object):                   
    """ Agent class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'id': 'non_defined'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.plans = list()
        

In [6]:
class Plan(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'selected': 'no'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.activities = list()


In [7]:
class Activity(object):
    """ Activities class """    
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'undefined',
            'link': None,
            'x': None,
            'y': None,
            'end_time': None
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))
        
        self.legs = list()
        

In [8]:
class Leg(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'include_routes': False,
            'mode': 'car'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        # To-Do ftm we leave MATSim to calculate initial routes
        if self.include_routes:
            self.routes = list()
        else:
            self.routes = None


In [9]:
class Route(object):
    """ Plans class """
    def __init__(self, **kwargs):
        prop_defaults = {
            'type': 'links'
        }

        for (prop, default) in prop_defaults.items():
            setattr(self, prop, kwargs.get(prop, default))

        self.value = None

# Inputs

## Zones

In [10]:
zone_centroids = pd.read_csv('../Network/Zones/zones_centroids_EPSG6312.csv', index_col=3)
zone_centroids = zone_centroids[['x', 'y']]
zone_centroids.head()

,x,y
NO,,
1,"532,581.66","3,892,173.91"
2,"532,426.63","3,892,226.77"
3,"532,114.61","3,892,154.10"
4,"532,030.41","3,892,609.90"
5,"531,746.95","3,892,600.84"


## Demand

In [11]:
mats = pd.read_csv('../Demand/matrices_demand.txt', delimiter=';', index_col=[0,1,2])
mats_info = pd.read_csv('../Demand/matrices_info.txt', delimiter=';', index_col=0)

C:\Users\Work\Miniconda3\envs\py3\lib\site-packages\numpy\lib\arraysetops.py:379: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [12]:
mats.head()

VALUE
MATRIXNO FROMNO TONO       
1        1      3000   3.00
                4000  17.00
                5000  11.00
                6000   2.00
         2      4000   1.00

In [13]:
mats_info.head()

,CODE,NAME,MATRIXTYPE,OBJECTTYPEREF,DSEGCODE,FILENAME,RANDOMROUND,NUMDECPLACES,DATASOURCETYPE,FORMULA
MATRIXNO,,,,,,,,,,
1,External_All,External_All,Demand,Zone,NaN,nan,0,3,Data,NaN
2,External_PrT,External_PrT,Demand,Zone,NaN,nan,0,3,Data,NaN
3,External_PuT,External_PuT,Demand,Zone,NaN,nan,0,3,Data,NaN
4,Goods,Goods,Demand,Zone,NaN,nan,0,3,Data,NaN
5,HBW,HBW,Demand,Zone,NaN,nan,0,3,Data,NaN


In [14]:
df = mats.join(mats_info.NAME)
df.reset_index(inplace=True)
df = df.drop('MATRIXNO', 1)
df['NAME'] = df['NAME'].astype('category')
df = df.set_index(['NAME', 'FROMNO', 'TONO'])
df.sort_index(inplace=True)

In [15]:
demand = df.loc[['HBW_C', 'HBW_X', 'HBEDU_C', 'HBEDU_X', 'HBO_C', 'HBO_X', 'HBSH_C', 'HBSH_X', 'NHB_C', 'NHB_X',],:]

In [16]:
# Cleaning
demand.reset_index(inplace=True)
demand = demand.assign(Purpose = lambda x: x.NAME.str.split('_').str[0])
demand = demand.assign(Direction = lambda x: x.NAME.str.split('_').str[1])
demand.drop('NAME', axis=1, inplace=True)

demand.rename(columns={'FROMNO': 'From_Node', 'TONO': 'To_Node', 'VALUE': 'Trips'}, inplace=True)
demand.Purpose.astype('category')
demand.Direction.astype('category')

demand.set_index(['Purpose', 'Direction', 'From_Node', 'To_Node'], inplace=True)

### Test demand

## 

In [17]:
demand_synthPop = demand.xs(['HBW', 'C'], level=[0,1], drop_level=False)

In [19]:
# Departure time
depTime = '08:00' # 8 hours
depTime_dev = '00:30' # deviation

In [20]:
# Create integer trips
# To-Do improve method
trips = demand_synthPop.Trips.round()

# exclude 0 trips
trips = trips[trips!=0]
# !!!!!!!!!!!!!!!!
trips = trips.head(10)

pop = []

agent_id=1
for trip in trips.iteritems():
    idx = trip[0]
    val = trip[1]
    # Dictionary with the trips info
    d = dict(zip(trips.index.names, idx))
    for t in range(0, int(val)):
        
        # Create the agent
        new_agent = Agent(id=agent_id)
        
        # Create the plans
        # we have only one plan
        new_plan = Plan(selected='yes')        
        
        # Create the activities
        # Get Node's coords
        x, y = zone_centroids.loc[d['From_Node']]
        end_time = randNormal_time(depTime, depTime_dev)
                
        # To-Do must generate random points in the zone
        new_act = Activity(type=d['Purpose'], x=x, y=y, end_time=end_time)
        
        # Create the leg
        new_leg = Leg(mode='car')
        
        new_act.legs.append(new_leg)
        new_plan.activities.append(new_act)
        new_agent.plans.append(new_plan)
        pop.append(new_agent)
        agent_id+=1
        

In [ ]:
def build_pop_xml(pop):
    import collections

    root = Element('population')
   
    xml_person = SubElement(root, 'person')
    xml_plan = SubElement(xml_person, 'plan')
    xml_activity = SubElement(xml_plan, 'act')
    xml_leg = SubElement(xml_plan, 'leg')
    xml_route = SubElement(xml_leg, 'route')

    d = {'person': xml_person,
         'plan': xml_plan,
         'activity': xml_activity,
         'leg': xml_leg,
         'route': xml_route}
    
    elems = collections.OrderedDict(d)

    
    for person in pop:
        expand_class(person)
            
            
    print(tostring(root))
#            for act in plans.activities:
#                for leg in act.legs:
#                    for route in leg.routes:
#                        
#        
#    for k1, elem in elems.items():
#        for k2, attr in attrs[k1].items():
#            elem.set(k2, attr)
#

In [47]:
def expand_class(c):
    for attr, v in c.__dict__.items():
        if isinstance(v, list):
            for e in v:
                myprint(e)
        else:
            
            print("{0} :: {1} ::: {2}".format(c.__class__.__name__, attr, v))


In [48]:
for person in pop:
    expand_class(person)

Agent :: id ::: 1
Plan :: selected ::: yes
Activity :: type ::: HBW
Activity :: link ::: None
Activity :: x ::: 532581.655663
Activity :: y ::: 3892173.90964
Activity :: end_time ::: 7:53:05
Leg :: include_routes ::: False
Leg :: mode ::: car
Leg :: routes ::: None
Agent :: id ::: 2
Plan :: selected ::: yes
Activity :: type ::: HBW
Activity :: link ::: None
Activity :: x ::: 532581.655663
Activity :: y ::: 3892173.90964
Activity :: end_time ::: 7:46:15
Leg :: include_routes ::: False
Leg :: mode ::: car
Leg :: routes ::: None
Agent :: id ::: 3
Plan :: selected ::: yes
Activity :: type ::: HBW
Activity :: link ::: None
Activity :: x ::: 532581.655663
Activity :: y ::: 3892173.90964
Activity :: end_time ::: 8:10:53
Leg :: include_routes ::: False
Leg :: mode ::: car
Leg :: routes ::: None
Agent :: id ::: 4
Plan :: selected ::: yes
Activity :: type ::: HBW
Activity :: link ::: None
Activity :: x ::: 532581.655663
Activity :: y ::: 3892173.90964
Activity :: end_time ::: 7:54:46
Leg :: incl

In [26]:
pop[0].plans[0].__dict__.items()

dict_items([('selected', 'yes'), ('activities', [<__main__.Activity object at 0x000001700AE9FA20>])])

In [ ]:


attrs = {}
attrs['person'] = {'id': agent_id} 
attrs['activity'] = {'type': activity_type, 
                        'x': coord_x,
                        'y': coord_y
                    }

attrs['plan'] = {'selected': '1'} 
attrs['leg'] = {'mode': mode} 


<population >
<person id= "1">
<plan selected= "yes " score= " 93.2987721 ">
<act type= " home " link= "1" end_time= " 07:16:23 " />
<leg mode= "car ">
<route type= " links ">1 2 3</ route >
</ leg >
<act type= " work " link= "3" end_time= " 17:38:34 " />
<leg mode= "car ">
<route type= " links ">3 1</ route >
</ leg >
<act type= " home " link= "1" />
</ plan >
</ person >
<person id= "2">
<plan selected= "yes " score= " 144.39002 ">
...
</ plan >
</ person >
</ population >

main_nodes = pd.read_csv(r'C:\Users\haris.ballis\OneDrive - Transport Systems Catapult\Personal\PhD\Model_Cyprus\Data\Network\Zones\nodes_in-fully-detailed-area.csv')
mn = main_nodes.NO.tolist()

idxsl = pd.IndexSlice
int_demand = demand.loc[idxsl[:, :, mn, mn], :]